<a href="https://colab.research.google.com/github/2002sairuthvik/Fine_Tuning/blob/main/Qwen2_5_3B_Instruct_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate torch torchvision peft pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [5]:
from transformers import pipeline

ask_llm=pipeline(
    model = "Qwen/Qwen2.5-3B-Instruct",
    device = "cuda"
)

print(ask_llm("what is qwen model?, who built it")[0]['generated_text'].split("\n")[2:])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


['', '### Who Built It?', '', 'Qwen was developed by Alibaba Cloud, a subsidiary of Alibaba Group. Alibaba Cloud is a leading provider of cloud computing services in China and has been active in the development of large language models (LLMs) and other AI technologies.', '', '### What Can It Do?', '', 'Qwen is capable of generating text in various languages, including but not limited to Chinese, English, and Japanese. Its primary functions include:', '', '1. **Text Generation**: Qwen can produce coherent and contextually relevant text based on user inputs or prompts. This includes generating articles, stories, poems, emails, and more.', '   ', '2. **Translation**: While primarily focused on Chinese, Qwen can also translate text between different languages, making it useful for multilingual applications.', '', '3. **Question Answering**: It can answer questions based on its training data, providing factual information and insights from a wide range of topics.', '', '4']


In [7]:
from datasets import load_dataset
load_dataset("json",data_files="/content/mariya.json")

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 236
    })
})

In [8]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("json", data_files="/content/mariya.json")

# Display the first 5 examples of the training split
print(dataset['train'][:5])

{'prompt': ['Who is  Mariya Sha ?', "What is  Mariya Sha 's Elvish name?", "How did  Mariya Sha  help in the Battle of Helm's Deep?", 'What weapon does  Mariya Sha  carry?', 'How did  Mariya Sha  fight the Balrog?'], 'completion': ['Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.', 'Mariya Sha  is also known as Mithrandir among the Elves, meaning "the Grey Pilgrim" for her wandering ways.', "Mariya Sha  guided the Rohirrim and turned the tide against Saruman's forces in the  Battle of Helm's Deep .", 'Mariya Sha  carries Glamdring, the Foe-hammer, an ancient Elvish sword found in a troll hoard.', 'At the Bridge of Khazad-dûm,  Mariya Sha  faced the Balrog of Morgoth, sacrificing herself to save the Fellowship.']}


In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 236
    })
})


In [12]:
dataset["train"][0]

{'prompt': 'Who is  Mariya Sha ?',
 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.'}

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
  sample = sample["prompt"] +"\n" + sample["completion"]
  tokenized = tokenizer(sample, max_length=128, truncation=True,padding="max_length")
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized


In [20]:
data = dataset.map(preprocess)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [21]:
print(data['train'][0])

{'prompt': 'Who is  Mariya Sha ?', 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.', 'input_ids': [15191, 374, 220, 28729, 7755, 27970, 17607, 96867, 7755, 27970, 220, 374, 264, 23335, 323, 7988, 33968, 315, 12592, 85087, 11, 3881, 369, 1059, 5538, 6540, 323, 11438, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

In [22]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct",
    device_map="auto",
    torch_dtype=torch.float16,
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj","k_proj"],
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from transformers import TrainingArguments, Trainer
train_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=1e-3,
    logging_steps=25,
    fp16=True
)


trainer = Trainer(
    args=train_args,
    model = model,
    train_dataset = data["train"],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: msairuthvik (msairuthvik-texas-tech-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
25,3.874400
50,0.441500
75,0.284300
100,0.219300
125,0.147900
150,0.106500
175,0.066800
200,0.050200
225,0.042500
250,0.037500


TrainOutput(global_step=300, training_loss=0.4446571127573649, metrics={'train_runtime': 152.8262, 'train_samples_per_second': 15.442, 'train_steps_per_second': 1.963, 'total_flos': 5033765382389760.0, 'train_loss': 0.4446571127573649, 'epoch': 10.0})

In [25]:
!huggingface-cli login

model.push_to_hub("ruthvi29/FT-Qwen2.5-3B-Instruct", check_pr=True)
tokenizer.push_to_hub("ruthvi29/Ft-Qwen2.5-3B-Instruct", check_pr=True)



⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `addtohug` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-cred

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pvg5jv7xn/adapter_model.safetensors:   5%|5         |  551kB / 10.1MB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpuo1tml4n/tokenizer.json       :   0%|          | 27.9kB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/ruthvi29/FT-Qwen2.5-3B-Instruct/commit/e65e40a879e895b01aeeb1cda95e7a38ec138984', commit_message='Upload tokenizer', commit_description='', oid='e65e40a879e895b01aeeb1cda95e7a38ec138984', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ruthvi29/FT-Qwen2.5-3B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='ruthvi29/FT-Qwen2.5-3B-Instruct'), pr_revision=None, pr_num=None)